# Declaração de Funções

In [1]:
import numpy as np, math # Para operações básicas com vetores e matrizes

def norma_vetor(vetor):
    soma_quadrados = 0.0

    # Calcula a soma dos quadrados dos elementos do vetor
    for elemento in vetor:
        soma_quadrados += elemento ** 2

    # Retorna a raiz quadrada da soma dos quadrados
    norma = math.sqrt(soma_quadrados)

    return norma

def gram_schmidt(matriz):
    """
    Realiza a decomposição QR de uma matriz matriz usando o processo de Gram-Schmidt.

    Args:
    - matriz: Matriz quadrada (numpy array)

    Returns:
    - Q: Matriz ortogonal (numpy array)
    - R: Matriz triangular superior (numpy array)
    """

    tam_matriz = matriz.shape[0]
    Q = np.zeros_like(matriz, dtype=float)
    R = np.zeros_like(matriz, dtype=float)

    for j in range(tam_matriz):
        vetor = matriz[:, j]

        print(f"vetor: {j+1}", vetor)

        for i in range(j):
            R[i, j] = np.dot(Q[:, i], matriz[:, j])
            vetor -= R[i, j] * Q[:, i]
        R[j, j] = norma_vetor(vetor)
        if R[j, j] > 1e-10:  # Evita divisão por zero
            Q[:, j] = vetor / R[j, j]
        else:
            Q[:, j] = vetor  # Vetor nulo, mantém como está

    return Q, R

def householder(A):
    """
    Realiza a decomposição QR de uma matriz usando reflexões de Householder.

    Args:
    - A: Matriz (numpy array)

    Returns:
    - Q: Matriz ortogonal (numpy array)
    - R: Matriz triangular superior (numpy array)
    """
    m, n = A.shape
    R = A.copy()
    Q = np.eye(m)

    for j in range(n):
        # Aplicando a reflexão de Householder para zerar os elementos abaixo da diagonal em R
        x = R[j:, j]
        norma_x = norma_vetor(x)
        v = x.copy()
        v[0] = x[0] + np.sign(x[0]) * norma_x
        if norma_vetor(v) != 0:
            v = v / norma_vetor(v)

        R[j:, j:] -= 2 * np.outer(v, np.dot(v.T, R[j:, j:]))

        Q[:, j:] -= 2 * np.outer(np.dot(Q[:, j:], v), v.T)

    return Q, R

def givens(A):
    m, n = A.shape
    Q = np.eye(m)
    R = A.copy()

    for j in range(n):
        for i in range(m-1, j, -1):
            if R[i, j] != 0:
                # Calcular os parâmetros da rotação de Givens
                r = np.sqrt(R[j, j]**2 + R[i, j]**2)
                c = R[j, j] / r
                s = -R[i, j] / r

                # Aplicar a rotação de Givens à direita de R
                G = np.identity(m)
                G[[j, i], [j, i]] = c
                G[j, i] = s
                G[i, j] = -s
                R = np.dot(G, R)

                # Acumular a matriz Q
                Q = np.dot(Q, G.T)

    return Q, R

def metodo_qr(matriz_inicial, metodo, max_iteracoes=100, erro=1e-8):
    """
    Aplica o método QR iterativo para encontrar autovalores de uma matriz matriz_inicial.

    Args:
    - matriz_inicial: Matriz quadrada (numpy array)
    - max_iteracoes: Número máximo de iterações (opcional)

    Returns:
    - autovalores: Lista de autovalores aproximados
    """

    A_k = matriz_inicial.copy()
    tam_matriz = matriz_inicial.shape[0]
    autovalores = []

    print(f"Matriz A_1:")
    print(A_k)

    for k in range(max_iteracoes):
        if metodo == "gs":
            Q, R = gram_schmidt(A_k)
        if metodo == "hh":
            Q, R = householder(A_k)
        if metodo == "rg":
            Q, R = givens(A_k)

        print(f"Matriz Q_{k+1}:")
        print(Q)
        print(f"Matriz R_{k+1}:")
        print(R)

        A_k = np.dot(R, Q) # Multiplicação RxQ
        print(f"Matriz A_{k+2}:")
        print(A_k)

        # Extração dos autovalores da diagonal de A_k
        autovalores_k = [A_k[i, i] for i in range(tam_matriz)]
        autovalores.append(autovalores_k)

        # Critério de parada (todos os elementos abaixo da diagonal principal são menores que o Erro)
        criterio_parada = True
        for i in range(1, tam_matriz):
            for j in range(i):
                if abs(A_k[i][j]) >= erro:
                    criterio_parada = False
                    break
            if not criterio_parada:
                break

        if criterio_parada:
            break

    return autovalores


# Exemplo passo a passo do funcionamento do código:


## Definindo a matrix inicial:

In [2]:
matriz_inicial_exemplo = np.array([
    [2, 3, 0],
    [0, 4, 0],
    [2, -3, 1]
], dtype=float)

## Iteração 1:

Na primeira iteração temos que A<sub>1</sub> = matrix inicial:

In [3]:
A1 = matriz_inicial_exemplo
A1

array([[ 2.,  3.,  0.],
       [ 0.,  4.,  0.],
       [ 2., -3.,  1.]])

Ao encontrar A<sub>1</sub>, fazemos a decomposição QR para encontrar as matrizes Q<sub>1</sub> e R<sub>1</sub>:

In [4]:
Q1, R1 = gram_schmidt(A1)
print("\nQ1:")
print(Q1)
print("\nR1:")
print(R1)

vetor: 1 [2. 0. 2.]
vetor: 2 [ 3.  4. -3.]
vetor: 3 [0. 0. 1.]

Q1:
[[ 0.70710678  0.51449576 -0.48507125]
 [ 0.          0.68599434  0.72760688]
 [ 0.70710678 -0.51449576  0.48507125]]

R1:
[[ 2.82842712  0.          0.70710678]
 [ 0.          5.83095189 -0.51449576]
 [ 0.          0.          0.48507125]]


## Iteração 2:

Ao encontrar as matrizes Q<sub>1</sub> e R<sub>1</sub>, é possível calcular a matrix A<sub>2</sub> = (R<sub>1</sub>)⋅(Q<sub>1</sub>)

In [5]:
A2 = np.dot(R1, Q1)
A2

array([[ 2.5       ,  1.09141031, -1.02899151],
       [-0.36380344,  4.26470588,  3.99307359],
       [ 0.34299717, -0.2495671 ,  0.23529412]])

Ao encontrar A<sub>2</sub>, fazemos a decomposição QR para encontrar as matrizes Q<sub>2</sub> e R<sub>2</sub>:

In [6]:
Q2, R2 = gram_schmidt(A2)
print("\nQ2:")
print(Q2)
print("\nR2:")
print(R2)

vetor: 1 [ 2.5        -0.36380344  0.34299717]
vetor: 2 [ 1.09141031  4.26470588 -0.2495671 ]
vetor: 3 [-1.02899151  3.99307359  0.23529412]

Q2:
[[ 0.98058068  0.15304883 -0.12262787]
 [-0.14269545  0.98573674  0.08922488]
 [ 0.13453456 -0.06999375  0.98843377]]

R2:
[[ 2.54950976  0.42808634 -1.54714743]
 [ 0.          4.3883845   3.76216429]
 [ 0.          0.          0.7150372 ]]


## Iteração 3:

Ao encontrar as matrizes Q<sub>2</sub> e R<sub>2</sub>, é possível calcular a matrix A<sub>3</sub> = (R<sub>2</sub>)⋅(Q<sub>2</sub>)

In [7]:
A3 = np.dot(R2, Q2)
A3

array([[ 2.23076923,  0.92047058, -1.80369776],
       [-0.12006138,  4.06246385,  4.11020333],
       [ 0.09619721, -0.05004814,  0.70676692]])

Assim o processo é executado iterativamente até que os valores abaixo da diagonal principal da matrix A<sub>k</sub>, onde k = iter_atual sejam menores que o **erro (critério de parada)**, ou que atinja a quantidade máxima de iterações. Os autovalores serão os elementos da diagonal principal da matriz A<sub>k</sub>.

# Execução do código

In [8]:
# Exemplo de uso:
if __name__ == "__main__":
    # Definindo uma matriz de exemplo
    matriz_inicial = np.array([
        [2, 3, 0],
        [0, 4, 0],
        [2, -3, 1]
    ], dtype=float)

## Metodo Gram-Schmidt:

In [9]:
autovalores_gs = metodo_qr(matriz_inicial, "gs", 3)
print("\nAutovalores aproximados: (Metodo de Gram-Schmidt)")
for i, autovalor in enumerate(autovalores_gs[-1]):
  print(f"lambda_{i+1} =", autovalor)

Matriz A_1:
[[ 2.  3.  0.]
 [ 0.  4.  0.]
 [ 2. -3.  1.]]
vetor: 1 [2. 0. 2.]
vetor: 2 [ 3.  4. -3.]
vetor: 3 [0. 0. 1.]
Matriz Q_1:
[[ 0.70710678  0.51449576 -0.48507125]
 [ 0.          0.68599434  0.72760688]
 [ 0.70710678 -0.51449576  0.48507125]]
Matriz R_1:
[[ 2.82842712  0.          0.70710678]
 [ 0.          5.83095189 -0.51449576]
 [ 0.          0.          0.48507125]]
Matriz A_2:
[[ 2.5         1.09141031 -1.02899151]
 [-0.36380344  4.26470588  3.99307359]
 [ 0.34299717 -0.2495671   0.23529412]]
vetor: 1 [ 2.5        -0.36380344  0.34299717]
vetor: 2 [ 1.09141031  4.26470588 -0.2495671 ]
vetor: 3 [-1.02899151  3.99307359  0.23529412]
Matriz Q_2:
[[ 0.98058068  0.15304883 -0.12262787]
 [-0.14269545  0.98573674  0.08922488]
 [ 0.13453456 -0.06999375  0.98843377]]
Matriz R_2:
[[ 2.54950976  0.42808634 -1.54714743]
 [ 0.          4.3883845   3.76216429]
 [ 0.          0.          0.7150372 ]]
Matriz A_3:
[[ 2.23076923  0.92047058 -1.80369776]
 [-0.12006138  4.06246385  4.11020333

## Metodo Householder:

In [10]:
autovalores_hh = metodo_qr(matriz_inicial, "hh", 3)

print("\nAutovalores aproximados: (Metodo de Householder)")
for i, autovalor in enumerate(autovalores_hh[-1]):
  print(f"lambda_{i+1} =", autovalor)

Matriz A_1:
[[ 2.  3.  0.]
 [ 0.  4.  0.]
 [ 2. -3.  1.]]
Matriz Q_1:
[[-0.70710678 -0.51449576  0.48507125]
 [ 0.         -0.68599434 -0.72760688]
 [-0.70710678  0.51449576 -0.48507125]]
Matriz R_1:
[[-2.82842712e+00  8.88178420e-16 -7.07106781e-01]
 [ 0.00000000e+00 -5.83095189e+00  5.14495755e-01]
 [ 2.22044605e-16  0.00000000e+00 -4.85071250e-01]]
Matriz A_2:
[[ 2.5         1.09141031 -1.02899151]
 [-0.36380344  4.26470588  3.99307359]
 [ 0.34299717 -0.2495671   0.23529412]]
Matriz Q_2:
[[-0.98058068 -0.15304883  0.12262787]
 [ 0.14269545 -0.98573674 -0.08922488]
 [-0.13453456  0.06999375 -0.98843377]]
Matriz R_2:
[[-2.54950976e+00 -4.28086345e-01  1.54714743e+00]
 [ 5.55111512e-17 -4.38838450e+00 -3.76216429e+00]
 [-5.55111512e-17  5.55111512e-17 -7.15037199e-01]]
Matriz A_3:
[[ 2.23076923  0.92047058 -1.80369776]
 [-0.12006138  4.06246385  4.11020333]
 [ 0.09619721 -0.05004814  0.70676692]]
Matriz Q_3:
[[-0.99763033 -0.054571    0.04190147]
 [ 0.05369308 -0.9983195  -0.02179991]


## Metodo Givens:

In [11]:
autovalores_rg = metodo_qr(matriz_inicial, "rg", 3)

print("\nAutovalores aproximados: (Metodo Rotação de Givens)")
for i, autovalor in enumerate(autovalores_rg[-1]):
    print(f"lambda_{i+1} =", autovalor)


Matriz A_1:
[[ 2.  3.  0.]
 [ 0.  4.  0.]
 [ 2. -3.  1.]]
Matriz Q_1:
[[ 0.70710678  0.          0.70710678]
 [ 0.          1.          0.        ]
 [-0.70710678  0.          0.70710678]]
Matriz R_1:
[[ 0.          4.24264069 -0.70710678]
 [ 0.          4.          0.        ]
 [ 2.82842712  0.          0.70710678]]
Matriz A_2:
[[ 0.5         4.24264069 -0.5       ]
 [ 0.          4.          0.        ]
 [ 1.5         0.          2.5       ]]
Matriz Q_2:
[[ 0.31622777 -0.67290046  0.66873386]
 [ 0.          0.70490738  0.70929937]
 [-0.9486833  -0.22430015  0.22291129]]
Matriz R_2:
[[-1.26491106  1.34164079 -2.52982213]
 [-0.67290046 -0.03524537 -0.22430015]
 [ 0.66873386  5.67439492  0.22291129]]
Matriz A_3:
[[ 2.00000000e+00  2.36433122e+00 -4.58189795e-01]
 [ 4.53343375e-17  4.78260870e-01 -5.24989873e-01]
 [-4.51413400e-17  3.49993249e+00  4.52173913e+00]]
Matriz Q_3:
[[ 1.00000000e+00  2.54317634e-17  1.94026053e-17]
 [-2.26671688e-17  1.35390384e-01  9.90792331e-01]
 [ 2.2570670

## Comparando os metodos:

In [12]:
print("\nAutovalores aproximados:")
print("           Metodo de Gram-Schmidt |   Metodo Householder   | Metodo Rotação de Givens")

# Pegue os autovalores da última iteração de cada método
autovalores_gs_ultima = autovalores_gs[-1]
autovalores_hh_ultima = autovalores_hh[-1]
autovalores_rg_ultima = autovalores_rg[-1]

# Itere sobre os autovalores de cada método simultaneamente
for i, (autovalor_gs, autovalor_hh, autovalor_rg) in enumerate(zip(autovalores_gs_ultima, autovalores_hh_ultima, autovalores_rg_ultima)):
    print(f"lambda_{i+1} = {autovalor_gs:.20f} | {autovalor_hh:.20f} | {autovalor_rg:.20f}")



Autovalores aproximados:
           Metodo de Gram-Schmidt |   Metodo Householder   | Metodo Rotação de Givens
lambda_1 = 2.10769230769230730971 | 2.10769230769230730971 | 1.99999999999999977796
lambda_2 = 4.02212224294963061055 | 4.02212224294963593962 | 4.04854995619899415971
lambda_3 = 0.87018544935805997032 | 0.87018544935805985929 | 0.95145004380100539620
